<a href="https://colab.research.google.com/github/seovalue/capstone-design-2021/blob/main/code/CNN_basic_with_2_layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 값을 고정
torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [4]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [7]:
# MNIS dataset
mnist_train = dsets.MNIST(root = "MNIST_data/",
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw




Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [8]:
mnist_test = dsets.MNIST(root = 'MNIST_data/',
                         train = False,
                         transform = transforms.ToTensor(),
                         download = True)

In [9]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                          batch_size = batch_size,
                                          shuffle = True,
                                          drop_last = True)

In [11]:
class CNN(nn.Module):

  def __init__(self):
    super(CNN,self).__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding =1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
    
    self.layer2 = nn.Sequential(
        nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )

    self.fc= nn.Linear(7*7*64, 10, bias=True) #한 줄로 펼쳤을 때, 결과값 10개
    torch.nn.init.xavier_uniform_(self.fc.weight)

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)

    out = out.view(out.size(0), -1) # 배치사이즈만큼 펼치고, 나머지는 한줄로
    out = self.fc(out)
    return out

In [12]:
model = CNN().to(device)

In [13]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [14]:
# training
total_batch = len(data_loader) #전체 배치 사이즈
for epoch in range(training_epochs):
  avg_cost = 0 #loss를 여기에 담을 것임!

  for X, Y in data_loader:
    X = X.to(device) #X는 이미지
    Y = Y.to(device) #Y는 레이블

    optimizer.zero_grad() # 이를 빼먹으면 학습이 안됨, 왜?
    hypothesis = model(X)
    
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch
  
  print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))

[Epoch:1] cost = 0.220241978764534
[Epoch:2] cost = 0.06092900037765503
[Epoch:3] cost = 0.04603453725576401
[Epoch:4] cost = 0.03669411689043045
[Epoch:5] cost = 0.030065080150961876
[Epoch:6] cost = 0.02619291841983795
[Epoch:7] cost = 0.020932242274284363
[Epoch:8] cost = 0.018487991765141487
[Epoch:9] cost = 0.01570901647210121
[Epoch:10] cost = 0.013570691458880901
[Epoch:11] cost = 0.01177073922008276
[Epoch:12] cost = 0.008641942404210567
[Epoch:13] cost = 0.00852913036942482
[Epoch:14] cost = 0.006149158347398043
[Epoch:15] cost = 0.007355728652328253


In [15]:
with torch.no_grad(): # 학습을 안할 것
  X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
  Y_test = mnist_test.test_labels.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy: ', accuracy.item())

Accuracy:  0.983299970626831


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:53: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
